In [9]:
"""
写了课件里的几个平滑算法：加一法，good-turing,Katz回退法，绝对减值，线性减值
加一法，good-turing,绝对减值，线性减值适用2gram。3gram时容易出现历史信息即前两个词的序列不存在概率为零的情况
Katz回退法可以3gram，回退用递归写的，3gram时算的比较慢。

对未登录词，事先随机选择一个频率为1-3的已经记录的词，用来替换未登录词

训练集corpus.txt. 用open(file, encoding='gb18030', errors='ignore')忽略错误
"""

"\n写了课件里的几个平滑算法：加一法，good-turing,Katz回退法，绝对减值，线性减值\n加一法，good-turing,绝对减值，线性减值适用2gram。3gram时容易出现历史信息即前两个词的序列不存在概率为零的情况\nKatz回退法可以3gram，回退用递归写的，3gram时算的比较慢。\n\n对未登录词，事先随机选择一个频率为1-3的已经记录的词，用来替换未登录词\n\n训练集corpus.txt. 用open(file, encoding='gb18030', errors='ignore')忽略错误\n"

In [23]:

import re
import random
from nltk import WordNetLemmatizer, pos_tag, word_tokenize
from collections import Counter
from nltk.corpus import wordnet

#处理句子，加n-1个"<BOS> "和" <EOS>"
def chuli(s, n): 

    # Convert to lowercases
    s = s.lower()
    # Replace all none alphanumeric characters with spaces
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    s = s.strip() #去掉换行符
    for i in range(n-1):  #加n-1个"<BOS> "和" <EOS>"
        s = "<BOS> " + s + " <EOS>"
    return s

#将句子切成n元词组
def ngram_generator(s, n): 
    # Break sentence in the token, remove empty tokens
    token = [token for token in s.split(" ") if token != ""]

    # Stemming and Lemmatizing
    lemmatizer = WordNetLemmatizer()
    tagged = pos_tag(token)
    token = []
    for word, tag in tagged:
        wntag = get_wordnet_pos(tag)
        if wntag is None:  # not supply tag in case of None
            lemma = lemmatizer.lemmatize(word)
            token.append(lemma)
        else:
            lemma = lemmatizer.lemmatize(word, pos=wntag)
            token.append(lemma)

    # Use the zip function to help us generate n-grams
    # Concatentate the tokens into ngrams and return
    ngrams = zip(*[token[i:] for i in range(n)])
    return ngrams

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

#读取文档，将n元词组存入字典
def diction(file, n, k): #参数k表示为每个句子增加（k-1)个<BOS>,<EOS>
    with open(file, encoding='gb18030', errors='ignore') as f: #忽略错误
        diction = Counter()
        for line in f:
            if line.strip() == "":
                continue
            line = chuli(line, k)
            sentence = ngram_generator(line, n)
            diction += Counter(sentence)
    return diction

#生成词组的频率数组fre，fre[i]储存i元词组的频率字典, 0<=i<=n
def cal_frequency(file,n):
    fre = []
    for i in range(1,n+1):
        word = diction(file, i, n)
        fre.append(word)
    num = sum(fre[0].values())
    fre = [{("num",):num}] + fre       
    return fre

#对一个词word,计算未平滑的概率
def unsmooth(word, n, frequency):
    count = frequency[n].get(word, 0)
    pre_word = word[:-1] if len(word[:-1]) >= 1 else ("num",)
    p = count / frequency[n-1][pre_word]
    return p
        
#对一个词word,计算加一法平滑的概率
def jiayifa(word, n, frequency):
    count = frequency[n].get(word, 0)
    N = len(frequency[1]) - (2 if n > 1 else 0)
    pre_word = word[:-1] if len(word[:-1]) >= 1 else ("num",)        
    p = (count + 1) / (frequency[n-1][pre_word] + N)
    return p

#对一个词word,计算good-turing平滑的概率
def good_turing(word, n, frequency):
    if n == 1:
        return unsmooth(word, 1, frequency)       
        
    pre_word = word[:-1] if len(word[:-1]) >= 1 else ("num",)
    num = frequency[n-1][pre_word]
    R_n = {} #记录出现次数为r的词个数
    for key, value in frequency[n].items():
        if key[:-1] == word[:-1]:
            R_n[value] = R_n.get(value, 0) + 1                     
    R_n[0] = len(frequency[1]) - sum(R_n.values()) #出现次数为0的词个数
    RtoRxing = {} #记录r对应的r*
    r_max = max(R_n.keys())
    r = 0
    while(r<r_max):
        if r in R_n:
            rj = r + 1
            while(rj not in R_n):
                rj += 1
            RtoRxing[r] = R_n[rj] * rj / R_n[r]
            r = rj
        else:
            r += 1
    RtoRxing[r_max] = r_max
    N = 0
    for key in R_n:
        N += R_n[key] * RtoRxing[key] #计算平滑后词的总数N
    rk = frequency[n].get(word, 0)
    p = RtoRxing[rk] / N
    return p

#对一个词word,计算Katz平滑的概率， Katz里面含递归，算的慢。
#Katz是回退法可以算3gram，可以处理3gram中前两个词排列未出现的情况。
def Katz(word, n, frequency):
    if n == 1:
        return unsmooth(word, 1, frequency) 
    if word in frequency[n]:
        return good_turing(word, n, frequency)
    else:
        beta = 0
        Na = 0
        wordin = []
        for word2 in frequency[n]:
            if word2[:-1] == word[:-1]:
                wordin.append(word2[-1])
                beta += good_turing(word2, n, frequency)
        for word3 in frequency[1]:
            if word3 not in wordin:
                pre_word = word[1:-1] + word3
                Na += Katz(pre_word, n - 1, frequency)
                              
        alpha = (1- beta) / Na
        return alpha * Katz(word[1:], n - 1, frequency)

#对一个词word,计算绝对减值法平滑的概率，b为自由参数0<b<=1
def absolute_discouting(word, n, frequency, b=0.4): #b为自由参数0<b<=1
    if n == 1:
        return unsmooth(word, 1, frequency)       
    
    N = frequency[n-1][word[:-1]]
    num = 0
    for key in frequency[n].keys():
        if key[:-1] == word[:-1]:
            num += 1
    if word in frequency[n]:
        p = (frequency[n][word] - b) / N
    else:
        p = b * num / (len(frequency[1]) - num) / N        
    return p

#对一个词word,计算线性减值法平滑的概率
def linear_discouting(word, n, frequency):
    if n == 1:
        return unsmooth(word, 1, frequency)       
    
    N = frequency[n-1][word[:-1]]
    num = 0
    n1 = 0
    for key in frequency[n].keys():
        if key[:-1] == word[:-1]:
            num += 1
            if frequency[n][key] == 1:
                n1 += 1
    alpha = n1 / N
    if word in frequency[n]:
        p = frequency[n][word] * (1 - alpha) / N
    else:
        p = alpha / (len(frequency[1]) - num)       
    return p
  
    

#事先随机选择一个频率为1-3的已经记录的词，用来替换未登录词
def select_oov(frequency): 
    oov = ('$$$$$$$',)
    for i in range(500):
        t = random.sample(frequency[1].keys(), 1)[0]
        if 1<= frequency[1][t] <= 3:
            oov = t
            break
    if oov == ('$$$$$$$',):
        oov = random.sample(frequency[1].keys(), 1)[0]
    return oov[0]
    
#计算句子s的概率， 加了try,历史信息即前n-1个词的排列在smooth频率数组中不存在时，概率为0.1 ** 20          
def calP(s, n,frequency, smooth, oov = ""):
    if oov == "":
        oov = select_oov(frequency)
    s = chuli(s, n)
    t = ngram_generator(s, n)
    P = 1
    sentence = []
    for word in t:
        
        word_ti = []#将word词组替换为word_ti
        for i in range(len(word)):
            if (word[i],) not in frequency[1]: #如果word[i]为未登录词，替换成oov
                t = oov
            else:
                t = word[i]
            word_ti.append(t)
        word_ti = tuple(word_ti)
        
        if(word != word_ti): #############中间结果，可去掉
            print(word,"计算时替换成",word_ti) #############中间结果，可去掉
            
        try:
            pk = smooth(word_ti, n, frequency)
        except:
            print("历史信息即前", str(n-1) ,"个词的排序",word_ti[:-1],"未在训练集里出现过")
            pk = 0.1 ** 20
        P *= pk
        sentence.append([word,pk])
    
    pinghua = {unsmooth:"未平滑",jiayifa:"加一法平滑",good_turing:"古德图灵平滑",
        absolute_discouting:"绝对减值平滑",linear_discouting:"线性减值平滑",Katz:"Kazt平滑"}
    
    print(sentence)   #############中间结果，可去掉
    print(s,pinghua[smooth], "概率是", P) #############中间结果，可去掉
    return P 




In [11]:
#测试

file = 'corpus.txt'
fre = cal_frequency(file,2) #2表示2gram
oov = select_oov(fre) #事先随机选择一个频率为1-3的已经记录的词，用来替换未登录词
s = " such as deep ssssssssss neural networks learning architectures"

In [12]:
p = calP(s, 2,fre, unsmooth, oov) #未平滑

('deep', 'sss') 计算时替换成 ('deep', 'ukrainian')
('sss', 'neural') 计算时替换成 ('ukrainian', 'neural')
[[('<BOS>', 'such'), 0.0021946564885496184], [('such', 'a'), 0.6013986013986014], [('a', 'deep'), 0.001204291657543245], [('deep', 'sss'), 0.0], [('sss', 'neural'), 0.0], [('neural', 'network'), 0.09722222222222222], [('network', 'learn'), 0.0], [('learn', 'architecture'), 0.0], [('architecture', '<EOS>'), 0.06666666666666667]]
<BOS> such as deep ssssssssss neural networks learning architectures <EOS> 未平滑 概率是 0.0


In [13]:
p = calP(s, 2,fre, jiayifa, oov) #加一法

('deep', 'sss') 计算时替换成 ('deep', 'ukrainian')
('sss', 'neural') 计算时替换成 ('ukrainian', 'neural')
[[('<BOS>', 'such'), 0.0007066721630057122], [('such', 'a'), 0.010831834720421563], [('a', 'deep'), 0.0003679175864606328], [('deep', 'sss'), 4.251519918370818e-05], [('sss', 'neural'), 4.258399693395222e-05], [('neural', 'network'), 0.00033964507090090854], [('network', 'learn'), 4.245923913043478e-05], [('learn', 'architecture'), 4.2468254979402894e-05], [('architecture', '<EOS>'), 0.0001275944198707043]]
<BOS> such as deep ssssssssss neural networks learning architectures <EOS> 加一法平滑 概率是 3.984325562425389e-34


In [14]:
p = calP(s, 2,fre, good_turing, oov) # good_turing

('deep', 'sss') 计算时替换成 ('deep', 'ukrainian')
('sss', 'neural') 计算时替换成 ('ukrainian', 'neural')
[[('<BOS>', 'such'), 0.001017915309446254], [('such', 'a'), 0.37554585152838427], [('a', 'deep'), 0.00117096018735363], [('deep', 'sss'), 2.2243312177471973e-05], [('sss', 'neural'), 2.129199846697611e-05], [('neural', 'network'), 0.1956521739130435], [('network', 'learn'), 1.45757485456642e-05], [('learn', 'architecture'), 1.2026747486409776e-05], [('architecture', '<EOS>'), 0.08108108108108109]]
<BOS> such as deep ssssssssss neural networks learning architectures <EOS> 古德图灵平滑 概率是 5.895435553325106e-28


In [24]:
p = calP(s, 2, fre, absolute_discouting, oov) #绝对减值

('deep', 'sss') 计算时替换成 ('deep', 'ukrainian')
('sss', 'neural') 计算时替换成 ('ukrainian', 'neural')
[[('<BOS>', 'such'), 0.0021564885496183207], [('such', 'a'), 0.6004662004662005], [('a', 'deep'), 0.0011604992336325815], [('deep', 'sss'), 1.2243327386574318e-05], [('sss', 'neural'), 1.703359877358089e-05], [('neural', 'network'), 0.09166666666666666], [('network', 'learn'), 8.77321245796169e-06], [('learn', 'architecture'), 8.396856426875188e-06], [('architecture', '<EOS>'), 0.05333333333333334]]
<BOS> such as deep ssssssssss neural networks learning architectures <EOS> 绝对减值平滑 概率是 1.1286830370497503e-28


In [20]:
p = calP(s, 2, fre, linear_discouting, oov) #线性减值

('deep', 'sss') 计算时替换成 ('deep', 'ukrainian')
('sss', 'neural') 计算时替换成 ('ukrainian', 'neural')
[[('<BOS>', 'such'), 0.0019094349105529982], [('such', 'a'), 0.4836422318939801], [('a', 'deep'), 0.0009485082312640798], [('deep', 'sss'), 2.6235701542659253e-05], [('sss', 'neural'), 4.258399693395222e-05], [('neural', 'network'), 0.08371913580246915], [('network', 'learn'), 1.644977335867817e-05], [('learn', 'architecture'), 1.4432096983691731e-05], [('architecture', '<EOS>'), 0.04]]
<BOS> such as deep ssssssssss neural networks learning architectures <EOS> 线性减值平滑 概率是 7.7800675605487655e-28


In [19]:
p = calP(s, 2,fre, Katz, oov)   #Katz回退法

('deep', 'sss') 计算时替换成 ('deep', 'ukrainian')
('sss', 'neural') 计算时替换成 ('ukrainian', 'neural')
[[('<BOS>', 'such'), 0.001017915309446254], [('such', 'a'), 0.37554585152838427], [('a', 'deep'), 0.00117096018735363], [('deep', 'sss'), 1.516396990205584e-06], [('sss', 'neural'), 0.00010463139232418534], [('neural', 'network'), 0.1956521739130435], [('network', 'learn'), 6.45668402000512e-05], [('learn', 'architecture'), 2.459284862320591e-05], [('architecture', '<EOS>'), 0.08108108108108109]]
<BOS> such as deep ssssssssss neural networks learning architectures <EOS> Kazt平滑 概率是 1.789016066731929e-27


In [25]:
file = 'corpus.txt'
fre = cal_frequency(file,3)

In [ ]:
p = calP(s, 3,fre, Katz, oov)#Katz回退法，里面有递归，3gram算地慢，笔记本跑一个小时没跑出来

('a', 'deep', 'sss') 计算时替换成 ('a', 'deep', 'ukrainian')
